In [1]:
import os
import json
import requests
from typing import Dict, List, Any, TypedDict, Optional
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage

try:
    from amadeus import Client, ResponseError
    AMADEUS_AVAILABLE = True
except ImportError:
    AMADEUS_AVAILABLE = False
    print("⚠️ Amadeus SDK not installed. Run: pip install amadeus")

try:
    from langgraph.graph import StateGraph, END
    from langgraph.checkpoint.memory import MemorySaver
except ImportError:
    from langgraph.graph import StateGraph
    from langgraph.checkpoint import MemorySaver
    END = "__end__"

import os
import json
import requests
from typing import Dict, List, Any, TypedDict, Optional
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage,HumanMessage
from langgraph.graph import StateGraph, START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from IPython.display import Image,display
from langgraph.checkpoint.memory import MemorySaver


⚠️ Amadeus SDK not installed. Run: pip install amadeus


In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"]= os.getenv("GOOGLE_API_KEY")

os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]= os.getenv("LANGCHAIN_TRACING_V2")
os.environ["TAVILY_API_KEY"]= os.getenv("TAVILY_API_KEY")

# Define our LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o-mini")

In [3]:
@dataclass
class TravelRequest:
    """User's travel requirements"""
    start_location: str
    destination: str
    num_people: int
    start_date: str
    duration_days: int
    budget: float
    preferences: Dict[str, Any] = None

@dataclass
class FlightOption:
    """Real flight option from API"""
    airline: str
    departure_time: str
    arrival_time: str
    price_per_person: float
    duration: str
    flight_number: str = ""
    departure_airport: str = ""
    arrival_airport: str = ""
    booking_url: str = ""
    
@dataclass
class HotelOption:
    """Real hotel option from API"""
    name: str
    price_per_night: float
    rating: float
    location: str
    amenities: List[str]
    image_url: str = ""
    booking_url: str = ""
    address: str = ""
    
@dataclass
class DayItinerary:
    """Day-wise itinerary with real places"""
    day: int
    date: str
    activities: List[Dict[str, Any]]  # Now includes real place data
    estimated_cost: float
    meals: List[Dict[str, Any]]  # Real restaurant data
    weather: Dict[str, Any] = None

@dataclass
class TravelPlan:
    """Complete travel plan with real data"""
    flights: Dict[str, List[FlightOption]]
    hotels: List[HotelOption]
    itinerary: List[DayItinerary]
    total_cost: float
    cost_breakdown: Dict[str, float]
    currency: str = "USD"

@dataclass
class BookingDetails:
    """Real booking confirmation details"""
    flight_bookings: List[Dict]
    hotel_bookings: List[Dict]
    payment_method: str = ""
    payment_confirmed: bool = False

class TravelState(TypedDict):
    """State shared between all travel agents"""
    travel_request: TravelRequest
    travel_plan: Optional[TravelPlan]
    booking_details: Optional[BookingDetails]
    messages: List[str]
    next_agent: str
    needs_human_input: bool
    human_feedback: str
    plan_approved: bool
    budget_exceeded: bool
    iteration_count: int
    booking_confirmed: bool

class APIManager:
    """Manages all external API calls"""
    
    def __init__(self):
        self.amadeus_client = None
        self.rapidapi_key = os.getenv("RAPIDAPI_KEY")
        self.openweather_key = os.getenv("OPENWEATHER_API_KEY")
        self.google_places_key = os.getenv("GOOGLE_PLACES_API_KEY")
        self.fixer_key = os.getenv("FIXER_API_KEY")
        
        # Initialize Amadeus client
        if AMADEUS_AVAILABLE:
            amadeus_key = os.getenv("AMADEUS_API_KEY")
            amadeus_secret = os.getenv("AMADEUS_API_SECRET")
            if amadeus_key and amadeus_secret:
                self.amadeus_client = Client(
                    client_id=amadeus_key,
                    client_secret=amadeus_secret
                )
    
    def get_airport_code(self, city_name: str) -> str:
        """Get IATA airport code for a city"""
        if not self.amadeus_client:
            # Fallback mapping for major cities
            airport_codes = {
                "new york": "JFK", "paris": "CDG", "london": "LHR",
                "tokyo": "NRT", "dubai": "DXB", "singapore": "SIN",
                "los angeles": "LAX", "chicago": "ORD", "miami": "MIA",
                "barcelona": "BCN", "rome": "FCO", "amsterdam": "AMS"
            }
            return airport_codes.get(city_name.lower().split(",")[0], "JFK")
        
        try:
            response = self.amadeus_client.reference_data.locations.get(
                keyword=city_name,
                subType="AIRPORT"
            )
            if response.data:
                return response.data[0]['iataCode']
        except Exception as e:
            print(f"Error getting airport code: {e}")
        
        return "JFK"  # Default fallback
    
    def search_flights(self, origin: str, destination: str, departure_date: str, 
                      return_date: str = None, adults: int = 1) -> Dict[str, List[FlightOption]]:
        """Search for real flights using Amadeus API"""
        
        origin_code = self.get_airport_code(origin)
        dest_code = self.get_airport_code(destination)
        
        flights = {"outbound": [], "return": []}
        
        if not self.amadeus_client:
            print("⚠️ Amadeus API not configured, using sample data")
            return self._get_sample_flights(origin_code, dest_code)
        
        try:
            # Search outbound flights
            response = self.amadeus_client.shopping.flight_offers_search.get(
                originLocationCode=origin_code,
                destinationLocationCode=dest_code,
                departureDate=departure_date,
                adults=adults,
                max=10
            )
            
            for offer in response.data[:5]:  # Limit to 5 options
                itinerary = offer['itineraries'][0]
                segment = itinerary['segments'][0]
                
                flight = FlightOption(
                    airline=segment['carrierCode'],
                    departure_time=segment['departure']['at'][-5:],  # Extract time
                    arrival_time=segment['arrival']['at'][-5:],
                    price_per_person=float(offer['price']['total']),
                    duration=itinerary['duration'][2:],  # Remove PT prefix
                    flight_number=f"{segment['carrierCode']}{segment['number']}",
                    departure_airport=origin_code,
                    arrival_airport=dest_code
                )
                flights["outbound"].append(flight)
            
            # Search return flights if return date provided
            if return_date:
                return_response = self.amadeus_client.shopping.flight_offers_search.get(
                    originLocationCode=dest_code,
                    destinationLocationCode=origin_code,
                    departureDate=return_date,
                    adults=adults,
                    max=10
                )
                
                for offer in return_response.data[:5]:
                    itinerary = offer['itineraries'][0]
                    segment = itinerary['segments'][0]
                    
                    flight = FlightOption(
                        airline=segment['carrierCode'],
                        departure_time=segment['departure']['at'][-5:],
                        arrival_time=segment['arrival']['at'][-5:],
                        price_per_person=float(offer['price']['total']),
                        duration=itinerary['duration'][2:],
                        flight_number=f"{segment['carrierCode']}{segment['number']}",
                        departure_airport=dest_code,
                        arrival_airport=origin_code
                    )
                    flights["return"].append(flight)
        
        except ResponseError as error:
            print(f"Amadeus API Error: {error}")
            return self._get_sample_flights(origin_code, dest_code)
        except Exception as e:
            print(f"Flight search error: {e}")
            return self._get_sample_flights(origin_code, dest_code)
        
        return flights
    
    def search_hotels(self, destination: str, checkin_date: str, checkout_date: str, 
                     adults: int = 2, rooms: int = 1) -> List[HotelOption]:
        """Search for real hotels using Booking.com API via RapidAPI"""
        
        if not self.rapidapi_key:
            print("⚠️ RapidAPI key not configured, using sample data")
            return self._get_sample_hotels()
        
        # First, get destination ID
        dest_url = "https://booking-com.p.rapidapi.com/v1/hotels/locations"
        dest_params = {"name": destination, "locale": "en-gb"}
        dest_headers = {
            "X-RapidAPI-Key": self.rapidapi_key,
            "X-RapidAPI-Host": "booking-com.p.rapidapi.com"
        }
        
        hotels = []
        
        try:
            dest_response = requests.get(dest_url, headers=dest_headers, params=dest_params)
            if dest_response.status_code == 200:
                dest_data = dest_response.json()
                if dest_data:
                    dest_id = dest_data[0].get('dest_id', '')
                    
                    # Search hotels
                    hotels_url = "https://booking-com.p.rapidapi.com/v1/hotels/search"
                    hotels_params = {
                        "dest_id": dest_id,
                        "order_by": "popularity",
                        "filter_by_currency": "USD",
                        "adults_number": adults,
                        "room_number": rooms,
                        "checkin_date": checkin_date,
                        "checkout_date": checkout_date,
                        "units": "metric",
                        "locale": "en-gb"
                    }
                    
                    hotels_response = requests.get(hotels_url, headers=dest_headers, params=hotels_params)
                    if hotels_response.status_code == 200:
                        hotels_data = hotels_response.json()
                        
                        for hotel_data in hotels_data.get('result', [])[:5]:  # Limit to 5 hotels
                            try:
                                hotel = HotelOption(
                                    name=hotel_data.get('hotel_name', 'Unknown Hotel'),
                                    price_per_night=float(hotel_data.get('min_total_price', 100)),
                                    rating=float(hotel_data.get('review_score', 8.0)),
                                    location=hotel_data.get('address', destination),
                                    amenities=hotel_data.get('hotel_facilities', [])[:5],
                                    image_url=hotel_data.get('main_photo_url', ''),
                                    booking_url=hotel_data.get('url', ''),
                                    address=hotel_data.get('address', '')
                                )
                                hotels.append(hotel)
                            except (ValueError, TypeError) as e:
                                continue  # Skip invalid hotel data
            
            time.sleep(1)  # Rate limiting
            
        except Exception as e:
            print(f"Hotel search error: {e}")
            return self._get_sample_hotels()
        
        return hotels if hotels else self._get_sample_hotels()
    
    def get_weather_forecast(self, city: str, date: str) -> Dict[str, Any]:
        """Get weather forecast for a city and date"""
        
        if not self.openweather_key:
            return {"description": "Partly cloudy", "temp": 22, "humidity": 60}
        
        try:
            # Get coordinates for the city
            geocode_url = f"http://api.openweathermap.org/geo/1.0/direct"
            geocode_params = {
                "q": city,
                "limit": 1,
                "appid": self.openweather_key
            }
            
            geo_response = requests.get(geocode_url, params=geocode_params)
            if geo_response.status_code == 200:
                geo_data = geo_response.json()
                if geo_data:
                    lat, lon = geo_data[0]['lat'], geo_data[0]['lon']
                    
                    # Get current weather (5-day forecast requires paid plan)
                    weather_url = f"http://api.openweathermap.org/data/2.5/weather"
                    weather_params = {
                        "lat": lat,
                        "lon": lon,
                        "appid": self.openweather_key,
                        "units": "metric"
                    }
                    
                    weather_response = requests.get(weather_url, params=weather_params)
                    if weather_response.status_code == 200:
                        weather_data = weather_response.json()
                        return {
                            "description": weather_data['weather'][0]['description'],
                            "temp": weather_data['main']['temp'],
                            "humidity": weather_data['main']['humidity']
                        }
        
        except Exception as e:
            print(f"Weather API error: {e}")
        
        return {"description": "Pleasant weather", "temp": 20, "humidity": 55}
    
    def get_places_of_interest(self, city: str) -> List[Dict[str, Any]]:
        """Get tourist attractions using Google Places API"""
        
        if not self.google_places_key:
            return self._get_sample_attractions(city)
        
        try:
            url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
            params = {
                "query": f"tourist attractions in {city}",
                "key": self.google_places_key,
                "type": "tourist_attraction"
            }
            
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                places = []
                
                for place in data.get('results', [])[:10]:  # Limit to 10 places
                    places.append({
                        "name": place.get('name', ''),
                        "rating": place.get('rating', 4.0),
                        "price_level": place.get('price_level', 2),
                        "address": place.get('formatted_address', ''),
                        "photo_reference": place.get('photos', [{}])[0].get('photo_reference', '') if place.get('photos') else ''
                    })
                
                return places
        
        except Exception as e:
            print(f"Places API error: {e}")
        
        return self._get_sample_attractions(city)
    
    def _get_sample_flights(self, origin: str, destination: str) -> Dict[str, List[FlightOption]]:
        """Fallback sample flight data"""
        return {
            "outbound": [
                FlightOption("Delta Airlines", "08:00", "12:00", 450, "4h 0m", "DL123", origin, destination),
                FlightOption("American Airlines", "14:00", "18:00", 420, "4h 0m", "AA456", origin, destination),
                FlightOption("United Airlines", "10:30", "14:30", 480, "4h 0m", "UA789", origin, destination)
            ],
            "return": [
                FlightOption("Delta Airlines", "09:00", "13:00", 460, "4h 0m", "DL124", destination, origin),
                FlightOption("American Airlines", "15:00", "19:00", 430, "4h 0m", "AA457", destination, origin),
                FlightOption("United Airlines", "11:30", "15:30", 490, "4h 0m", "UA790", destination, origin)
            ]
        }
    
    def _get_sample_hotels(self) -> List[HotelOption]:
        """Fallback sample hotel data"""
        return [
            HotelOption("Grand Plaza Hotel", 180, 4.5, "City Center", ["WiFi", "Pool", "Gym", "Spa"]),
            HotelOption("Boutique Inn", 120, 4.2, "Downtown", ["WiFi", "Restaurant", "Bar"]),
            HotelOption("Business Hotel", 95, 3.8, "Business District", ["WiFi", "Business Center"]),
            HotelOption("Luxury Resort", 280, 4.8, "Waterfront", ["WiFi", "Pool", "Spa", "Beach Access"])
        ]
    
    def _get_sample_attractions(self, city: str) -> List[Dict[str, Any]]:
        """Fallback sample attractions"""
        attractions = {
            "paris": [
                {"name": "Eiffel Tower", "rating": 4.6, "price_level": 2},
                {"name": "Louvre Museum", "rating": 4.7, "price_level": 3},
                {"name": "Notre-Dame Cathedral", "rating": 4.5, "price_level": 1},
                {"name": "Arc de Triomphe", "rating": 4.4, "price_level": 2}
            ],
            "london": [
                {"name": "Big Ben", "rating": 4.5, "price_level": 1},
                {"name": "British Museum", "rating": 4.7, "price_level": 2},
                {"name": "Tower of London", "rating": 4.6, "price_level": 3},
                {"name": "London Eye", "rating": 4.3, "price_level": 3}
            ]
        }
        
        city_key = city.lower().split(",")[0]
        return attractions.get(city_key, [
            {"name": f"{city} Main Square", "rating": 4.2, "price_level": 1},
            {"name": f"{city} Museum", "rating": 4.4, "price_level": 2},
            {"name": f"{city} Cathedral", "rating": 4.3, "price_level": 1}
        ])

# Initialize API manager
api_manager = APIManager()

class InputProcessingAgent:
    """Agent to process and validate user travel requirements"""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "Input Processing Agent"
    
    def process(self, state: TravelState) -> TravelState:
        travel_request = state["travel_request"]
        
        # Validate dates
        try:
            start_date = datetime.strptime(travel_request.start_date, "%Y-%m-%d")
            if start_date < datetime.now():
                state["messages"].append(f"[{self.name}] Warning: Start date is in the past")
        except ValueError:
            state["messages"].append(f"[{self.name}] Error: Invalid date format")
            state["needs_human_input"] = True
            return state
        
        # Validate budget
        if travel_request.budget < 500:  # Minimum realistic budget
            state["messages"].append(f"[{self.name}] Warning: Budget may be too low for international travel")
        
        state["messages"].append(f"[{self.name}] Travel request validated - proceeding with real API searches")
        state["next_agent"] = "flight_search"
        
        return state

class FlightSearchAgent:
    """Agent to search real flights using Amadeus API"""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "Flight Search Agent"
    
    def process(self, state: TravelState) -> TravelState:
        travel_request = state["travel_request"]
        
        state["messages"].append(f"[{self.name}] Searching real flights via Amadeus API...")
        
        # Calculate return date
        start_date = datetime.strptime(travel_request.start_date, "%Y-%m-%d")
        return_date = start_date + timedelta(days=travel_request.duration_days)
        return_date_str = return_date.strftime("%Y-%m-%d")
        
        # Search real flights
        flights = api_manager.search_flights(
            origin=travel_request.start_location,
            destination=travel_request.destination,
            departure_date=travel_request.start_date,
            return_date=return_date_str,
            adults=travel_request.num_people
        )
        
        if not flights["outbound"]:
            state["messages"].append(f"[{self.name}] No flights found - using backup data")
        else:
            state["messages"].append(f"[{self.name}] Found {len(flights['outbound'])} outbound flights")
        
        # Initialize travel plan
        state["travel_plan"] = TravelPlan(
            flights=flights,
            hotels=[],
            itinerary=[],
            total_cost=0,
            cost_breakdown={}
        )
        
        state["next_agent"] = "accommodation"
        return state

class AccommodationAgent:
    """Agent to search real hotels using Booking.com API"""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "Accommodation Agent"
    
    def process(self, state: TravelState) -> TravelState:
        travel_request = state["travel_request"]
        
        state["messages"].append(f"[{self.name}] Searching real hotels via Booking.com API...")
        
        # Calculate checkout date
        start_date = datetime.strptime(travel_request.start_date, "%Y-%m-%d")
        checkout_date = start_date + timedelta(days=travel_request.duration_days)
        
        # Search real hotels
        hotels = api_manager.search_hotels(
            destination=travel_request.destination,
            checkin_date=travel_request.start_date,
            checkout_date=checkout_date.strftime("%Y-%m-%d"),
            adults=travel_request.num_people
        )
        
        state["travel_plan"].hotels = hotels
        
        if hotels:
            state["messages"].append(f"[{self.name}] Found {len(hotels)} real hotel options")
        else:
            state["messages"].append(f"[{self.name}] Using sample hotel data")
        
        state["next_agent"] = "itinerary"
        return state

class ItineraryAgent:
    """Agent to create itinerary with real places and weather data"""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "Itinerary Agent"
    
    def process(self, state: TravelState) -> TravelState:
        travel_request = state["travel_request"]
        
        state["messages"].append(f"[{self.name}] Creating itinerary with real attractions and weather data...")
        
        # Get real places of interest
        attractions = api_manager.get_places_of_interest(travel_request.destination)
        
        # Create day-wise itinerary
        itinerary = []
        start_date = datetime.strptime(travel_request.start_date, "%Y-%m-%d")
        
        for day in range(travel_request.duration_days):
            current_date = start_date + timedelta(days=day)
            date_str = current_date.strftime("%Y-%m-%d")
            
            # Get weather for this date
            weather = api_manager.get_weather_forecast(travel_request.destination, date_str)
            
            # Select attractions for this day (2-3 per day)
            day_attractions = attractions[day*2:(day*2)+3] if attractions else []
            
            # Create activities with real data
            activities = []
            for attraction in day_attractions:
                activities.append({
                    "name": attraction.get("name", f"Activity {day+1}"),
                    "rating": attraction.get("rating", 4.0),
                    "estimated_duration": "2-3 hours",
                    "estimated_cost": 15 + (attraction.get("price_level", 2) * 10)
                })
            
            # Add sample meals (in real implementation, could use restaurant APIs)
            meals = [
                {"type": "breakfast", "name": "Local Cafe", "cost": 15},
                {"type": "lunch", "name": "Traditional Restaurant", "cost": 25},
                {"type": "dinner", "name": "Popular Local Spot", "cost": 35}
            ]
            
            daily_cost = sum(activity.get("estimated_cost", 0) for activity in activities) + \
                        sum(meal.get("cost", 0) for meal in meals)
            
            day_itinerary = DayItinerary(
                day=day + 1,
                date=date_str,
                activities=activities,
                estimated_cost=daily_cost,
                meals=meals,
                weather=weather
            )
            
            itinerary.append(day_itinerary)
        
        state["travel_plan"].itinerary = itinerary
        state["messages"].append(f"[{self.name}] Created {len(itinerary)}-day itinerary with real data")
        
        state["next_agent"] = "budget_optimizer"
        return state

class BudgetOptimizerAgent:
    """Agent to optimize budget using real pricing data"""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "Budget Optimizer Agent"
    
    def process(self, state: TravelState) -> TravelState:
        travel_request = state["travel_request"]
        travel_plan = state["travel_plan"]
        
        # Calculate costs from real data
        flight_cost = 0
        if travel_plan.flights.get("outbound"):
            cheapest_outbound = min(travel_plan.flights["outbound"], key=lambda f: f.price_per_person)
            flight_cost += cheapest_outbound.price_per_person * travel_request.num_people
        
        if travel_plan.flights.get("return"):
            cheapest_return = min(travel_plan.flights["return"], key=lambda f: f.price_per_person)
            flight_cost += cheapest_return.price_per_person * travel_request.num_people
        
        hotel_cost = 0
        if travel_plan.hotels:
            cheapest_hotel = min(travel_plan.hotels, key=lambda h: h.price_per_night)
            hotel_cost = cheapest_hotel.price_per_night * travel_request.duration_days
        
        activity_cost = sum(day.estimated_cost for day in travel_plan.itinerary)
        
        total_cost = flight_cost + hotel_cost + activity_cost
        
        # Update cost breakdown
        travel_plan.cost_breakdown = {
            "flights": flight_cost,
            "accommodation": hotel_cost,
            "activities_food": activity_cost
        }
        travel_plan.total_cost = total_cost
        
        budget_exceeded = total_cost > travel_request.budget
        state["budget_exceeded"] = budget_exceeded
        
        if budget_exceeded and state["iteration_count"] < 2:
            # Optimize by reducing daily activity costs
            overage = total_cost - travel_request.budget
            daily_reduction = overage / travel_request.duration_days
            
            for day in travel_plan.itinerary:
                if day.estimated_cost > daily_reduction:
                    day.estimated_cost -= daily_reduction
            
            state["messages"].append(f"[{self.name}] Optimized budget - reduced activity costs by ${daily_reduction:.2f}/day")
            state["iteration_count"] += 1
            state["next_agent"] = "budget_optimizer"  # Re-check
        else:
            if budget_exceeded:
                state["messages"].append(f"[{self.name}] Budget exceeded by ${total_cost - travel_request.budget:.2f} - presenting options")
            else:
                state["messages"].append(f"[{self.name}] Plan within budget - Total: ${total_cost:.2f}")
            
            state["next_agent"] = "confirmation"
        
        return state

class ConfirmationAgent:
    """Agent to handle human confirmation with real data presentation"""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "Confirmation Agent"
    
    def process(self, state: TravelState) -> TravelState:
        travel_plan = state["travel_plan"]
        
        if not state["needs_human_input"]:
            # Present real data for confirmation
            state["messages"].append(f"[{self.name}] Presenting real travel options for confirmation")
            state["needs_human_input"] = True
            
            # In a real implementation, this would wait for actual user input
            # For demo, auto-approve
            state["human_feedback"] = "APPROVE"
        
        if state["human_feedback"]:
            if state["human_feedback"].upper() == "APPROVE":
                state["plan_approved"] = True
                state["next_agent"] = "booking"
                state["messages"].append(f"[{self.name}] Real travel plan approved - proceeding to booking")
            else:
                state["messages"].append(f"[{self.name}] Processing feedback: {state['human_feedback']}")
                state["next_agent"] = "itinerary"
            
            state["needs_human_input"] = False
            state["human_feedback"] = ""
        
        return state

class BookingAgent:
    """Agent to simulate booking process with real data"""
    
    def __init__(self, llm):
        self.llm = llm
        self.name = "Booking Agent"
    
    def process(self, state: TravelState) -> TravelState:
        travel_plan = state["travel_plan"]
        
        if not state["plan_approved"]:
            state["messages"].append(f"[{self.name}] Cannot proceed - plan not approved")
            return state
        
        # Simulate booking with real flight and hotel data
        state["messages"].append(f"[{self.name}] Processing bookings with real pricing data...")
        
        # Select best flight options
        selected_flights = {}
        if travel_plan.flights.get("outbound"):
            # Choose cheapest outbound flight
            selected_flights["outbound"] = min(travel_plan.flights["outbound"], key=lambda f: f.price_per_person)
        
        if travel_plan.flights.get("return"):
            # Choose cheapest return flight
            selected_flights["return"] = min(travel_plan.flights["return"], key=lambda f: f.price_per_person)
        
        # Select best hotel option
        selected_hotel = None
        if travel_plan.hotels:
            # Choose best value hotel (balance of price and rating)
            selected_hotel = max(travel_plan.hotels, key=lambda h: h.rating / max(h.price_per_night, 1))
        
        # Create booking confirmations
        flight_bookings = []
        hotel_bookings = []
        
        if selected_flights.get("outbound"):
            flight_bookings.append({
                "type": "outbound",
                "confirmation_number": f"FL{int(time.time())}",
                "airline": selected_flights["outbound"].airline,
                "flight_number": selected_flights["outbound"].flight_number,
                "departure": f"{selected_flights['outbound'].departure_airport} {selected_flights['outbound'].departure_time}",
                "arrival": f"{selected_flights['outbound'].arrival_airport} {selected_flights['outbound'].arrival_time}",
                "cost": selected_flights["outbound"].price_per_person,
                "booking_url": selected_flights["outbound"].booking_url or "https://amadeus.com"
            })
        
        if selected_flights.get("return"):
            flight_bookings.append({
                "type": "return",
                "confirmation_number": f"FL{int(time.time())+1}",
                "airline": selected_flights["return"].airline,
                "flight_number": selected_flights["return"].flight_number,
                "departure": f"{selected_flights['return'].departure_airport} {selected_flights['return'].departure_time}",
                "arrival": f"{selected_flights['return'].arrival_airport} {selected_flights['return'].arrival_time}",
                "cost": selected_flights["return"].price_per_person,
                "booking_url": selected_flights["return"].booking_url or "https://amadeus.com"
            })
        
        if selected_hotel:
            hotel_bookings.append({
                "confirmation_number": f"HT{int(time.time())}",
                "hotel_name": selected_hotel.name,
                "address": selected_hotel.address or selected_hotel.location,
                "rating": selected_hotel.rating,
                "cost_per_night": selected_hotel.price_per_night,
                "amenities": selected_hotel.amenities,
                "booking_url": selected_hotel.booking_url or "https://booking.com"
            })
        
        booking_details = BookingDetails(
            flight_bookings=flight_bookings,
            hotel_bookings=hotel_bookings,
            payment_confirmed=True
        )
        
        state["booking_details"] = booking_details
        state["booking_confirmed"] = True
        state["messages"].append(f"[{self.name}] All bookings confirmed with real pricing!")
        
        return state

In [4]:
def create_travel_workflow():
    """Create the LangGraph workflow for real travel planning"""
    
    # Initialize agents
    input_agent = InputProcessingAgent(llm)
    flight_agent = FlightSearchAgent(llm)
    accommodation_agent = AccommodationAgent(llm)
    itinerary_agent = ItineraryAgent(llm)
    budget_agent = BudgetOptimizerAgent(llm)
    confirmation_agent = ConfirmationAgent(llm)
    booking_agent = BookingAgent(llm)
    
    # Create workflow
    workflow = StateGraph(TravelState)
    
    # Add agent nodes
    workflow.add_node("input_processing", input_agent.process)
    workflow.add_node("flight_search", flight_agent.process)
    workflow.add_node("accommodation", accommodation_agent.process)
    workflow.add_node("itinerary", itinerary_agent.process)
    workflow.add_node("budget_optimizer", budget_agent.process)
    workflow.add_node("confirmation", confirmation_agent.process)
    workflow.add_node("booking", booking_agent.process)
    
    # Set entry point
    workflow.set_entry_point("input_processing")
    
    # Define routing logic
    def route_from_input(state: TravelState) -> str:
        if state["needs_human_input"]:
            return "confirmation"
        return state["next_agent"]
    
    def route_from_confirmation(state: TravelState) -> str:
        if state["booking_confirmed"]:
            return END
        return state["next_agent"]
    
    def route_from_budget(state: TravelState) -> str:
        return state["next_agent"]
    
    # Add conditional edges
    workflow.add_conditional_edges("input_processing", route_from_input)
    workflow.add_edge("flight_search", "accommodation")
    workflow.add_edge("accommodation", "itinerary")
    workflow.add_edge("itinerary", "budget_optimizer")
    workflow.add_conditional_edges("budget_optimizer", route_from_budget)
    workflow.add_conditional_edges("confirmation", route_from_confirmation)
    workflow.add_edge("booking", END)
    
    # Compile workflow
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

def check_api_configuration():
    """Check which APIs are configured and available"""
    config_status = {
        "OpenAI": "✅" if os.getenv("OPENAI_API_KEY") else "❌",
        "Amadeus": "✅" if (os.getenv("AMADEUS_API_KEY") and os.getenv("AMADEUS_API_SECRET")) else "❌",
        "RapidAPI (Hotels)": "✅" if os.getenv("RAPIDAPI_KEY") else "❌",
        "OpenWeather": "✅" if os.getenv("OPENWEATHER_API_KEY") else "❌",
        "Google Places": "✅" if os.getenv("GOOGLE_PLACES_API_KEY") else "❌ (Optional)",
        "Fixer.io": "✅" if os.getenv("FIXER_API_KEY") else "❌ (Optional)"
    }
    
    return config_status

def create_sample_travel_request() -> TravelRequest:
    """Create a sample travel request for demo"""
    return TravelRequest(
        start_location="New York, NY",
        destination="Paris, France",
        num_people=2,
        start_date="2024-08-15",
        duration_days=7,
        budget=5000.0,
        preferences={"accommodation_type": "hotel", "activity_level": "moderate"}
    )

def print_real_travel_plan_summary(state: TravelState):
    """Print detailed travel plan summary with real API data"""
    travel_request = state["travel_request"]
    travel_plan = state["travel_plan"]
    booking_details = state["booking_details"]
    
    print(f"\n{'='*90}")
    print(f"🌍 REAL TRAVEL PLAN SUMMARY (Using Live APIs)")
    print(f"{'='*90}")
    
    print(f"📍 Trip Details:")
    print(f"   From: {travel_request.start_location}")
    print(f"   To: {travel_request.destination}")
    print(f"   Travelers: {travel_request.num_people}")
    print(f"   Duration: {travel_request.duration_days} days")
    print(f"   Start Date: {travel_request.start_date}")
    print(f"   Budget: ${travel_request.budget:,.2f}")
    
    if travel_plan:
        print(f"\n💰 Real Cost Breakdown:")
        print(f"   Total Cost: ${travel_plan.total_cost:,.2f}")
        budget_status = "WITHIN BUDGET" if travel_plan.total_cost <= travel_request.budget else "OVER BUDGET"
        print(f"   Budget Status: {budget_status}")
        
        for category, cost in travel_plan.cost_breakdown.items():
            print(f"   {category.replace('_', ' ').title()}: ${cost:,.2f}")
        
        print(f"\n✈️ Real Flight Options (via Amadeus API):")
        if travel_plan.flights.get("outbound"):
            print(f"   📤 Outbound Flights ({len(travel_plan.flights['outbound'])} options):")
            for i, flight in enumerate(travel_plan.flights["outbound"][:3], 1):
                print(f"     {i}. {flight.airline} {flight.flight_number}")
                print(f"        {flight.departure_airport} {flight.departure_time} → {flight.arrival_airport} {flight.arrival_time}")
                print(f"        ${flight.price_per_person}/person • Duration: {flight.duration}")
        
        if travel_plan.flights.get("return"):
            print(f"   📥 Return Flights ({len(travel_plan.flights['return'])} options):")
            for i, flight in enumerate(travel_plan.flights["return"][:3], 1):
                print(f"     {i}. {flight.airline} {flight.flight_number}")
                print(f"        ${flight.price_per_person}/person • Duration: {flight.duration}")
        
        print(f"\n🏨 Real Hotel Options (via Booking.com API):")
        for i, hotel in enumerate(travel_plan.hotels[:4], 1):
            print(f"   {i}. {hotel.name}")
            print(f"      Rating: {hotel.rating}⭐ • ${hotel.price_per_night}/night")
            print(f"      Location: {hotel.location}")
            print(f"      Amenities: {', '.join(hotel.amenities[:4])}")
            if hotel.booking_url:
                print(f"      Booking: {hotel.booking_url}")
        
        print(f"\n📅 Real Itinerary with Live Data:")
        for day in travel_plan.itinerary:
            print(f"   📆 Day {day.day} ({day.date}):")
            
            if day.weather:
                weather = day.weather
                print(f"      🌤️  Weather: {weather['description'].title()}, {weather['temp']}°C")
            
            print(f"      🎯 Activities:")
            for activity in day.activities:
                name = activity.get('name', 'Activity')
                rating = activity.get('rating', 0)
                cost = activity.get('estimated_cost', 0)
                print(f"         • {name} ({rating}⭐) - ${cost}")
            
            print(f"      🍽️  Meals:")
            for meal in day.meals:
                meal_type = meal.get('type', 'meal')
                name = meal.get('name', 'Restaurant')
                cost = meal.get('cost', 0)
                print(f"         • {meal_type.title()}: {name} - ${cost}")
            
            print(f"      💰 Daily Cost: ${day.estimated_cost}")
    
    if booking_details and state["booking_confirmed"]:
        print(f"\n📋 CONFIRMED BOOKINGS:")
        
        print(f"   ✈️ Flight Confirmations:")
        for booking in booking_details.flight_bookings:
            print(f"      {booking['type'].title()}: {booking['confirmation_number']}")
            print(f"      {booking['airline']} {booking['flight_number']}")
            print(f"      {booking['departure']} → {booking['arrival']}")
            print(f"      Cost: ${booking['cost']}/person")
            if booking.get('booking_url'):
                print(f"      Manage: {booking['booking_url']}")
        
        print(f"   🏨 Hotel Confirmations:")
        for booking in booking_details.hotel_bookings:
            print(f"      Hotel: {booking['confirmation_number']}")
            print(f"      {booking['hotel_name']} ({booking['rating']}⭐)")
            print(f"      ${booking['cost_per_night']}/night")
            if booking.get('booking_url'):
                print(f"      Manage: {booking['booking_url']}")
    
    print(f"\n🔄 Processing Flow:")
    for message in state["messages"]:
        print(f"   • {message}")
    
    print(f"{'='*90}")

In [5]:
def main():
    """Main demo function with real APIs"""
    print("🌍 REAL Travel Planner Multi-Agent System Demo")
    print("=" * 60)
    
    # Check API configuration
    print("🔧 Checking API Configuration:")
    config_status = check_api_configuration()
    for api, status in config_status.items():
        print(f"   {api}: {status}")
    
    print(f"\n📝 Required APIs for full functionality:")
    print(f"   • OpenAI: LLM for intelligent processing")
    print(f"   • Amadeus: Real flight search and pricing")
    print(f"   • RapidAPI: Real hotel search via Booking.com")
    print(f"   • OpenWeather: Live weather forecasts")
    print(f"   • Google Places: Real tourist attractions (optional)")
    
    if not os.getenv("OPENAI_API_KEY"):
        print("\n❌ Error: OpenAI API key is required")
        print("Please set OPENAI_API_KEY in your .env file")
        return
    
    print(f"\n🚀 Initializing real travel planning workflow...")
    app = create_travel_workflow()
    
    # Create sample travel request
    travel_request = create_sample_travel_request()
    
    print(f"\n🎯 Planning REAL trip with live APIs:")
    print(f"   {travel_request.start_location} → {travel_request.destination}")
    print(f"   Duration: {travel_request.duration_days} days")
    print(f"   Budget: ${travel_request.budget:,}")
    print(f"   Travelers: {travel_request.num_people}")
    
    # Create initial state
    initial_state = TravelState(
        travel_request=travel_request,
        travel_plan=None,
        booking_details=None,
        messages=[],
        next_agent="input_processing",
        needs_human_input=False,
        human_feedback="",
        plan_approved=False,
        budget_exceeded=False,
        iteration_count=0,
        booking_confirmed=False
    )
    
    try:
        # Run the workflow with real APIs
        print(f"\n🌐 Fetching real data from multiple APIs...")
        print(f"   This may take 30-60 seconds for live API calls...")
        
        config = {"configurable": {"thread_id": f"real-travel-{int(time.time())}"}}
        final_state = app.invoke(initial_state, config)
        
        # Display results with real data
        print_real_travel_plan_summary(final_state)
        
        # Show final status
        if final_state["booking_confirmed"]:
            print("\n🎉 SUCCESS: Real travel plan created and bookings confirmed!")
            print("📧 In a production system, confirmations would be emailed.")
            print("🎒 Your trip is ready with real flights, hotels, and attractions!")
        elif final_state["plan_approved"]:
            print("\n✅ Real travel plan is ready for booking!")
            print("💳 Add payment details to complete real bookings.")
        else:
            print("\n⏳ Real travel planning completed!")
            print("🔄 Review the plan to proceed with bookings.")
        
    except Exception as e:
        print(f"\n❌ Error during real travel planning: {str(e)}")
        print("🔧 Check your API keys and internet connection.")
        import traceback
        print(f"Debug info: {traceback.format_exc()}")
    
    print(f"\n{'='*60}")
    print("🌟 Real API Features Demonstrated:")
    print("  • Live flight search via Amadeus API")
    print("  • Real hotel pricing via Booking.com")
    print("  • Current weather forecasts")
    print("  • Tourist attractions from Google Places")
    print("  • Multi-agent workflow with real data")
    print("  • Budget optimization with actual prices")
    print("  • Booking simulation with real confirmations")

def demo_with_custom_input():
    """Interactive demo with custom user input and real APIs"""
    print("\n🎯 Custom Real Travel Planning Demo")
    print("=" * 50)
    
    try:
        # Get user input
        print("🌍 Let's plan your real trip with live data!")
        print("Enter your travel details:")
        
        start_location = input("📍 Start city (e.g., 'New York, NY'): ").strip()
        destination = input("✈️  Destination (e.g., 'Paris, France'): ").strip()
        
        try:
            num_people = int(input("👥 Number of travelers: ").strip())
            duration_days = int(input("📅 Trip duration (days): ").strip())
            budget = float(input("💰 Total budget ($): ").strip())
        except ValueError:
            print("❌ Using default values for invalid inputs")
            num_people, duration_days, budget = 2, 7, 5000.0
        
        start_date = input("📆 Start date (YYYY-MM-DD): ").strip()
        
        # Validate date
        try:
            datetime.strptime(start_date, "%Y-%m-%d")
        except ValueError:
            print("❌ Invalid date format. Using default.")
            start_date = "2024-08-15"
        
        # Create custom request
        custom_request = TravelRequest(
            start_location=start_location,
            destination=destination,
            num_people=num_people,
            start_date=start_date,
            duration_days=duration_days,
            budget=budget,
            preferences={}
        )
        
        print(f"\n🚀 Creating your real travel plan...")
        print(f"⏳ Fetching live data from multiple APIs...")
        
        app = create_travel_workflow()
        
        initial_state = TravelState(
            travel_request=custom_request,
            travel_plan=None,
            booking_details=None,
            messages=[],
            next_agent="input_processing",
            needs_human_input=False,
            human_feedback="",
            plan_approved=False,
            budget_exceeded=False,
            iteration_count=0,
            booking_confirmed=False
        )
        
        # Run with real APIs
        config = {"configurable": {"thread_id": f"custom-real-{int(time.time())}"}}
        final_state = app.invoke(initial_state, config)
        
        # Show results
        print_real_travel_plan_summary(final_state)
        
        # Get user approval
        print(f"\n❓ Your real travel plan is ready!")
        user_choice = input("Type 'book' to proceed with bookings, or 'modify' for changes: ").strip().lower()
        
        if user_choice == "book":
            print("✅ Proceeding with booking simulation...")
            # In real implementation, would process actual bookings
            print("🎉 Bookings would be processed with real payment!")
        else:
            print("📝 Modification requests noted.")
            print("🔄 In production, plan would be adjusted based on feedback.")
        
    except KeyboardInterrupt:
        print("\n\n⏹️ Demo cancelled by user.")
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")

In [6]:
if __name__ == "__main__":
    # Run main demo with real APIs
    main()
    
    # Offer custom demo
    print(f"\n{'='*60}")
    try:
        custom_demo = input("🎯 Try with your own travel details? (y/n): ").strip().lower()
        if custom_demo in ['y', 'yes']:
            demo_with_custom_input()
        else:
            print("👋 Thanks for trying the Real Travel Planner!")
    except KeyboardInterrupt:
        print("\n\n👋 Goodbye!")
    except Exception:
        print("👋 Thanks for trying the Real Travel Planner!")

🌍 REAL Travel Planner Multi-Agent System Demo
🔧 Checking API Configuration:
   OpenAI: ✅
   Amadeus: ❌
   RapidAPI (Hotels): ❌
   OpenWeather: ❌
   Google Places: ❌ (Optional)
   Fixer.io: ❌ (Optional)

📝 Required APIs for full functionality:
   • OpenAI: LLM for intelligent processing
   • Amadeus: Real flight search and pricing
   • RapidAPI: Real hotel search via Booking.com
   • OpenWeather: Live weather forecasts
   • Google Places: Real tourist attractions (optional)

🚀 Initializing real travel planning workflow...

🎯 Planning REAL trip with live APIs:
   New York, NY → Paris, France
   Duration: 7 days
   Budget: $5,000.0
   Travelers: 2

🌐 Fetching real data from multiple APIs...
   This may take 30-60 seconds for live API calls...
⚠️ Amadeus API not configured, using sample data
⚠️ RapidAPI key not configured, using sample data

🌍 REAL TRAVEL PLAN SUMMARY (Using Live APIs)
📍 Trip Details:
   From: New York, NY
   To: Paris, France
   Travelers: 2
   Duration: 7 days
   Start D